<a href="https://colab.research.google.com/github/dBenazzi/trash-can/blob/main/programmazione_di_applicazioni_data_intensive/Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio: Recommendation

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
`nome.cognome@unibo.it`

## Setup

- Importare le librerie necessarie, già usate nelle scorse esercitazioni

In [1]:
import numpy as np
import pandas as pd

- Nella seconda parte utilizzeremo la libreria **Surprise**
- Se si utilizza Colab (consigliato), installare Surprise con `pip`
  - eseguire la cella sotto rimuovendo `#`

In [2]:
pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3195813 sha256=f40955b7e802d62a3a5fe170b6ecea69739a943be128ee2aa39d10365bf90aba
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


- Se si utilizza un ambiente Anaconda, installare Surprise con `conda`
  - eseguire la cella sotto rimuovendo `#`

In [3]:
# conda install -c conda-forge scikit-surprise

- In altri casi installare con `pip` come sopra
  - può essere richiesta la compilazione dei sorgenti, accertarsi di aver installato un compilatore C e le librerie Python (es. su Debian e derivati `sudo apt install build-essentials python3-dev`)
- Una volta installato Surprise, eseguirne l'import per verifica

In [4]:
import surprise

## Recommendation e collaborative filtering

- I sistemi di _recommendation_ sono usati in molti contesti per suggerire agli **utenti** di un servizio alcuni degli **oggetti** offerti, in modo mirato secondo i loro interessi
  - suggerire prodotti da acquistare su Amazon
  - suggerire film o serie da vedere su Netflix
  - suggerire canzoni da ascoltare su Spotify
  - ...
- I metodi di _collaborative filtering_ forniscono suggerimenti sulla base delle **associazioni** esistenti tra utenti e oggetti
  - in genere vengono usati i **voti** dati dagli utenti agli oggetti
  - non vengono usate informazioni specifiche su singoli utenti (es. età) e oggetti (es. genere)

## Recommendation su dati Amazon

- Nelle prime esercitazioni avevamo visto un metodo di recommendation semplice applicato su dati di vendite estratti da Amazon
  - i dati riguardavano solamente i prodotti acquistati dai clienti, senza considerare i voti
- Riprendiamo ora gli stessi dati, ma includendo l'informazione sui **voti dati dai clienti**
- I dati sono divisi in due file CSV
  - uno contenente i voti raccolti fino alla fine del 2000, da usare come training set
  - uno contenente i voti raccolti dal 2001 in poi, da usare come validation set
- Scarichiamo i due file

In [5]:
import os.path
from urllib.request import urlretrieve
if not os.path.exists("amazon_train.csv"):
    urlretrieve("https://bit.ly/2LdmgTR", "amazon_train.csv")
if not os.path.exists("amazon_val.csv"):
    urlretrieve("https://bit.ly/2IPlyxO", "amazon_val.csv")

- Ciascun file CSV contiene tre colonne, nell'ordine:
  - codice e nome dell'utente
  - codice e nome del prodotto (film in VHS)
  - voto assegnato, da 1 a 5 stelle
- Carichiamo il file di training in un frame pandas con `read_csv`
  - `header=None` indica che il file **non** contiene una riga di intestazione
  - con `names` indichiamo manualmente i nomi delle colonne

In [6]:
amazon_train = pd.read_csv(
    "amazon_train.csv",
    header=None,
    names=["user", "item", "rating"],
)

- Vediamo i dati caricati

In [7]:
amazon_train.head(10)

,user,item,rating
0,[624547] Roland E. Zwick,[96429] Varsity Blues [VHS],2
1,[624547] Roland E. Zwick,[96654] Miss Julie [VHS],3
2,[1201103] Benjamin J Burgraff,[47474] The Last Starfighter [VHS],4
3,"[81819] dsrussell ""greyhater""",[95902] Meet Joe Black [VHS],3
4,[1092996] Reviewer,[43720] Honeymoon in Vegas [VHS],3
5,[1169585] Brooke276,[43871] Mccabe &amp; Mrs. Miller [VHS],5
6,[652570] Lawrance M. Bernabo,[101985] Little Women [VHS],5
7,[1092996] Reviewer,[100268] American Dreamer [VHS],4
8,[891040] casualsuede,[101569] Better Off Dead [VHS],3
9,"[125883] ""flickjunkie""",[55450] Mr Smith Goes to Washington [VHS],5


### Esercizio 1: Esplorazione frame

- **(1a)** Quanti sono i voti dati?
- **(1b)** Quanti sono gli utenti distinti nei dati?
- **(1c)** Quanti sono gli oggetti distinti?
- **(1d)** Qual'è la media di tutti i voti?
- **(1e)** Qual è il numero minimo di voti dato da un utente?
- **(1f)** Qual è l'oggetto con più voti?
- **(1g)** Quali sono i 10 oggetti col voto medio maggiore?
  - estrarre una serie con i nomi come etichette e i voti medi come valori

In [8]:
print("a: ", amazon_train.count())
print("b: ", amazon_train["user"].nunique())
print("c: ", amazon_train["item"].nunique())
print("d: ", amazon_train["rating"].mean())
print("e: ", amazon_train["user"].value_counts().min())
print("f: ", amazon_train["item"].value_counts().idxmax())
print("g: ", amazon_train.groupby("item")["rating"].mean().sort_values(ascending=False).head(10))

a:  user      9683
item      9683
rating    9683
dtype: int64
b:  178
c:  3384
d:  3.931219663327481
e:  30
f:  [57372] The Sixth Sense [VHS]
g:  item
[100004] High School Confidential [VHS]               5.0
[47499] Send Me No Flowers [VHS]                      5.0
[49068] Doctor Who - Revenge of the Cybermen [VHS]    5.0
[49062] The Duchess and the Dirtwater Fox [VHS]       5.0
[49036] Manhattan Melodrama [VHS]                     5.0
[48982] The King and I [VHS]                          5.0
[48922] A Summer Place [VHS]                          5.0
[48910] Blood Alley [VHS]                             5.0
[48874] Revenge of the Nerds [VHS]                    5.0
[48832] Enemy Below [VHS]                             5.0
Name: rating, dtype: float64


## Estrazione matrice voti

- Per lavorare agevolmente con i dati, rappresentiamoli come matrice dove
  - ogni **riga** corrisponde ad un **utente** $u$
  - ogni **colonna** corrisponde ad un **oggetto** $i$
  - ogni **cella** contiene il **voto** dato da $u$ a $i$, che può essere mancate (`NaN`)
- Per iniziare, impostiamo come indice del frame la coppia di colonne `user` e `item`

In [9]:
amazon_train.set_index(["user", "item"], inplace=True)

In [10]:
amazon_train.head(5)

rating
user                          item                                      
[624547] Roland E. Zwick      [96429] Varsity Blues [VHS]              2
                              [96654] Miss Julie [VHS]                 3
[1201103] Benjamin J Burgraff [47474] The Last Starfighter [VHS]       4
[81819] dsrussell "greyhater" [95902] Meet Joe Black [VHS]             3
[1092996] Reviewer            [43720] Honeymoon in Vegas [VHS]         3

- Utilizziamo quindi l'operazione di pivoting `unstack` sull'unica colonna rimasta `rating` per portare gli oggetti dall'indice delle righe a quello delle colonne

In [11]:
train_ratings = amazon_train["rating"].unstack("item")

In [12]:
train_ratings.iloc[:5, :5]

item,[100004] High School Confidential [VHS],[100019] Julius Caesar [VHS],[100027] Love Happy [VHS],[100028] Dark Mirror [VHS],[100066] Foul Play [VHS]
user,,,,,
[1001561] D. Wetzel,NaN,NaN,NaN,NaN,NaN
"[1001958] Neural Clone ""zarran67""",NaN,NaN,NaN,NaN,NaN
[1003282] CHI-TOWN,NaN,NaN,NaN,NaN,NaN
[1007279] James L.,NaN,NaN,NaN,4.0,NaN
"[1012166] Bertin Ramirez ""justareviewer""",NaN,NaN,NaN,NaN,NaN


- Per i prossimi passaggi, rappresentiamo questo frame in forma di matrice "semplice" che chiamiamo **R**, inserendo 0 al posto dei voti mancanti
  - $r_{u,i}$ = voto dato dall'utente u all'oggetto i

In [13]:
R = train_ratings.fillna(0).values

In [14]:
R[:4, :5]

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 4., 0.]])

- Estraiamo anche una matrice booleana **P** che indichi per quali coppie utente-prodotto esiste un voto
  - $p_{u,i}$ = 1 (`True`) se l'utente u ha dato un voto all'oggetto i, 0 (`False`) altrimenti

In [15]:
P = train_ratings.notna().values

In [16]:
P[:4, :5]

array([[False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False,  True, False]])

### Esercizio 2: Esplorazione matrici

Estrarre le stesse informazioni richieste mell'esercizio 1, ma utilizzando le matrici `R` e `P` invece dei frame
- **(2a)** Quanti sono i voti dati?
- **(2b)** Quanti sono gli utenti distinti nei dati?
- **(2c)** Quanti sono gli oggetti distinti?
- **(2d)** Qual'è la media di tutti i voti?
  - attenzione a non includere i voti mancanti (nulli) nella media
- **(2e)** Qual è il numero minimo di voti dato da un utente?
- **(2f)** Qual è l'oggetto con più voti
- **(2g)** Quali sono i 10 oggetti col voto medio maggiore?

- Per i punti 2f e 2g recuperare i nomi degli oggetti dai nomi delle colonne di `train_ratings`
- Per i punti 2d e 2g è utile usare il parametro `where` dei metodi di aggregazione (`sum`, `mean`, ...), che filtra i dati su cui eseguire l'operazione
  - disponibile solo in versioni recenti di NumPy

In [17]:
print("a: ", P.sum())
print("b: ", R.shape[0])
print("c: ", R.shape[1])
print("d: ", R.mean(where=P))
print("e: ", P.sum(axis=1).min())
print("f: ", train_ratings.columns[P.sum(axis=0).argmax()])
print("g: ", train_ratings.columns[(-R.mean(axis=0, where=P)).argsort()[:10]])

a:  9683
b:  178
c:  3384
d:  3.931219663327481
e:  30
f:  [57372] The Sixth Sense [VHS]
g:  Index(['[100004] High School Confidential [VHS]',
       '[49062] The Duchess and the Dirtwater Fox [VHS]',
       '[49036] Manhattan Melodrama [VHS]', '[48982] The King and I [VHS]',
       '[48922] A Summer Place [VHS]', '[48910] Blood Alley [VHS]',
       '[48874] Revenge of the Nerds [VHS]', '[48832] Enemy Below [VHS]',
       '[4880] Children of Heaven [VHS]',
       '[49068] Doctor Who - Revenge of the Cybermen [VHS]'],
      dtype='object', name='item')


## User-based collaborative filtering

- Nella recommendation _user-based_, il voto $\hat{r}_{u,i}$ previsto per un oggetto $i$ da parte di un utente $u$ è determinato in base ai **voti dati da altri utenti** ad $i$, pesati in base alla loro _similarità_ con $u$
- La similarità $\text{sim}(u,v)$ tra due utenti $u$ e $v$ è misurata dai **voti dati ad oggetti recensiti da entrambi**
- Iniziamo definendo come è calcolata la misura di similarità $\text{sim}$

## Similarità coseno

- La _similarità coseno_ è in generale una misura della similarità tra due vettori
  - è il coseno dell'angolo formato tra i due vettori
  - tiene conto del loro orientamento, non della lunghezza
  - è pari al prodotto scalare dei vettori suddiviso per il prodotto delle loro norme euclidee
  - è compresa tra 0 e 1 per vettori con valori non negativi
$$ \text{sim}(\mathbf{a},\mathbf{b}) = \frac{\sum_{i=1}^n a_i\cdot b_i}{\sqrt{\sum_{i=1}^n a_i^2}\cdot\sqrt{\sum_{i=1}^n b_i^2}} $$
- Nel collaborative filtering, possiamo usare la similarità coseno per comparare i voti dati da due utenti
  - consideriamo solo l'insieme di oggetti $C_{u,v}$ votati da entrambi
$$ \text{sim}(u,v) = \frac{\sum_{i\in C_{u,v}} r_{u,i}\cdot r_{v,i}}{\sqrt{\sum_{i\in C_{u,v}} r_{u,i}^2}\cdot\sqrt{\sum_{i\in C_{u,v}} r_{v,i}^2}}$$

- Costruiamo una matrice con le similarità coseno tra tutte le coppie di utenti usando opportune operazioni tra matrici
- Il numeratore della formula è una somma di prodotti, rappresentabile quindi come prodotto scalare
  - i termini da moltiplicare sono nelle righe della matrice **R** relative agli utenti _u_ e _v_
  - vanno contati solo gli oggetti votati da entrambi, ma essendo i voti mancanti 0 gli altri oggetti sono esclusi implicitamente
- Una matrice di prodotti scalari si può ottenere tramite un prodotto canonico tra matrici
  - dobbiamo ottenere una matrice in cui alla posizione _u, v_ troviamo il prodotto scalare tra `R[u, :]` e `R[v, :]`
  - possiamo ottenerla dal prodotto tra `R` e la sua trasposta  
  `(R @ R.T)[u, v] == R[u, :] @ R.T[:, v] == R[u, :] @ R[v, :]`

In [18]:
cosim_numer = R @ R.T

- Per il denominatore, dobbiamo calcolare le norme dei vettori dei voti di ciascun utente u, ma limitate agli acquisti in comune con ciascun altro utente v
- Iniziamo creando un array booleano 3d $\mathbf{P}^\text{AND}$ che, per qualsiasi coppia di utenti u, v e oggetto i, indichi se sia u che v hanno acquistato i
$$ p^\text{AND}_{u,v,i} = p_{u,i} \wedge p_{v,i} $$
- Per farlo, eseguiamo un AND tra diverse viste della matrice **P** con assi aggiunti e sfruttiamo le regole di broadcasting di NumPy

In [19]:
# M = numero utenti, N = numero oggetti
# P ha forma M x N

#          M x 1 x N       1 x M x N
P_and = P[:, None, :] & P[None, :, :]

# il risultato ha forma M x M x N
# P_and[u, v, i] == P[u, i] & P[v, i]

- Creiamo ora un array 3d $\mathbf{R}^\text{COM}$ che riporti i voti degli utenti solo per oggetti in comune con altri
  - $r^\text{COM}_{u,v,i} = r_{u,i}$ se sia u che v hanno dato un voto a i
  - $r^\text{COM}_{u,v,i} = 0$ altrimenti
  - ovvero:
$$ r^\text{COM}_{u,v,i} = p^\text{AND}_{u,v,i} \cdot r_{u,i} $$

In [20]:
#     M x M x N    M x 1 x N
R_com = P_and * R[:, None, :]

# risultato: M x M x N

- Da quì possiamo ottenere una matrice 2d che per ogni coppia u, v di utenti contiene il valore delle norme nel denominatore della formula della sim. coseno
$$ \forall u,v: \sqrt{\sum_{i\in C_{u,v}} r_{u,i}^2} = \sqrt{\sum_i (r^\text{COM}_{u,v,i})^2} $$
- Possiamo usare la funzione `norm` di NumPy per calcolare le norme lungo l'asse specificato (il terzo, quello indicizzato con $i$ nella formula sopra)

In [21]:
R_com_norms = np.linalg.norm(R_com, axis=2)

- Utilizziamo la matrice `R_com_norms` sopra per estrarre una matrice `cosim_denum` col denominatore completo della formula

In [22]:
cosim_denum = R_com_norms * R_com_norms.T

- Estraiamo infine la matrice `cosim` con tutte le similarità per ogni coppia di utenti

In [23]:
cosim = cosim_numer / cosim_denum

<ipython-input-23-a06e1fe2b8d7>:1: RuntimeWarning: invalid value encountered in true_divide
  cosim = cosim_numer / cosim_denum


- Nel calcolo di `cosim` riceviamo un warning per via di divisioni per zero, in corrispondenza delle quali troviamo valori mancanti (`nan`)

In [24]:
cosim[:4, :4]

array([[1.        , 0.9924812 , 0.94372216,        nan],
       [0.9924812 , 1.        , 0.97580525,        nan],
       [0.94372216, 0.97580525, 1.        ,        nan],
       [       nan,        nan,        nan, 1.        ]])

- Questi corrispondono a coppie di utenti senza acquisti in comune
- Usiamo la funzione `isnan` per localizzare i valori mancanti e li impostiamo a 0 (similarità nulla)

In [25]:
cosim[np.isnan(cosim)] = 0

In [26]:
cosim[:4, :4]

array([[1.        , 0.9924812 , 0.94372216, 0.        ],
       [0.9924812 , 1.        , 0.97580525, 0.        ],
       [0.94372216, 0.97580525, 1.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        ]])

## Predizione dei voti mancanti

- Vediamo ora come predire il voto $\hat{r}_{u,i}$ che un utente $u$ darebbe ad un oggetto $i$ che non conosce
- Come prima soluzione, prendiamo la **media dei voti** dati ad $i$ da qualsiasi altro utente $v$ che l'abbia valutato, **pesata in base alla similarità** tra $u$ e $v$
$$ \hat{r}_{u,i} = \frac{\sum_{v:P_{v,i}=1} \text{sim}(u, v) \cdot r_{v,i}}{\sum_{v:P_{v,i}=1} \text{sim}(u, v)} $$

### Esercizio 3: Calcolo predizioni

Definire la funzione `predict_from_all` in modo che, dati gli indici di un utente `u` e un oggetto `i`, restituisca il voto predetto
- `voters` è una lista di indici degli utenti che hanno già acquistato l'oggetto, da usare per selezionare righe e colonne nel punto successivo
  - il metodo `nonzero` è usato sul vettore booleano `P[:, i]` per restituire gli indici corrispondenti ai valori `True`
- impostare `predicted_vote` al voto predetto secondo la formula sopra
- restituire tale voto se non è un valore NaN (verificare con `np.isnan`), altrimenti restituire come ripiego la media di tutti i voti noti

In [27]:
def predict_from_all(u, i):
    voters = list(P[:, i].nonzero()[0])
    predicted_vote = (cosim[u, voters] @ R[voters, i]) / cosim[u, voters].sum()
    return predicted_vote if not np.isnan(predicted_vote) else R.mean(where=P)

## Validazione delle predizioni

- Una volta generati dei voti predetti, dobbiamo verificarne la bontà confrontandoli con voti veri
  - analogamente ai problemi di regressione, ci serve un _validation set_ di voti per la verifica disgiunto dal _training set_ su cui abbiamo costruito le predizioni
- Carichiamo i voti contenuti nel file `amazon_val.csv` da usare come validation set

In [28]:
amazon_val = pd.read_csv(
    "amazon_val.csv",
    header=None,
    names=["user", "item", "rating"],
)

- Come sopra, estraiamo i voti in forma di frame con una riga per ogni utente e una colonna per ogni oggetto

In [29]:
val_ratings = amazon_val.set_index(["user", "item"])["rating"].unstack("item")

In [30]:
val_ratings.iloc[:4, :4]

item,[100004] High School Confidential [VHS],[100019] Julius Caesar [VHS],[100027] Love Happy [VHS],[100066] Foul Play [VHS]
user,,,,
[1001561] D. Wetzel,NaN,NaN,NaN,NaN
[1003282] CHI-TOWN,NaN,NaN,NaN,NaN
[1007279] James L.,NaN,NaN,NaN,NaN
"[1012166] Bertin Ramirez ""justareviewer""",NaN,NaN,NaN,NaN


- Attenzione: le righe e colonne di questo frame non sono conformi a quello analogo dei dati di training
  - già la forma dei frame è diversa, perché alcuni utenti e oggetti presenti nel training non lo sono quì

In [31]:
val_ratings.shape == train_ratings.shape

False

- Utilizziamo il metodo `reindex_like` per ottenere un frame che contenga i dati di `val_ratings` ma con le stesse righe e colonne di `train_ratings`

In [32]:
val_ratings = val_ratings.reindex_like(train_ratings)

- Otteniamo così un nuovo frame compatibile con quello di training

In [33]:
val_ratings.iloc[:5, :5]

item,[100004] High School Confidential [VHS],[100019] Julius Caesar [VHS],[100027] Love Happy [VHS],[100028] Dark Mirror [VHS],[100066] Foul Play [VHS]
user,,,,,
[1001561] D. Wetzel,NaN,NaN,NaN,NaN,NaN
"[1001958] Neural Clone ""zarran67""",NaN,NaN,NaN,NaN,NaN
[1003282] CHI-TOWN,NaN,NaN,NaN,NaN,NaN
[1007279] James L.,NaN,NaN,NaN,NaN,NaN
"[1012166] Bertin Ramirez ""justareviewer""",NaN,NaN,NaN,NaN,NaN


In [34]:
np.array_equal(train_ratings.index, val_ratings.index)

True

In [35]:
np.array_equal(train_ratings.columns, val_ratings.columns)

True

- Analogamente a prima, estraiamo
  - una matrice dei voti `R_val`, impostando a 0 quelli mancanti
  - una matrice booleana `P_val` che indichi per quali coppie utente-oggetto è presente un voto

In [36]:
R_val = val_ratings.fillna(0).values
P_val = val_ratings.notna().values

In [37]:
R_val[:4, -4:]

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 5., 0., 4.]])

In [38]:
P_val[:4, -4:]

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       [False,  True, False,  True]])

- Estraiamo un vettore `val_actual` con tutti i voti definiti nel validation set

In [39]:
val_actual = R_val[P_val]

- Definiamo quindi una funzione `get_val_predictions` che
  - accetti in ingresso una funzione `pred_func(u, i)` che predice il voto di un utente `u` ad un oggetto `i`
  - la applichi su tutte le coppie del validation set
  - restituisca un vettore con le predizioni estratte, allineate ai voti reali del vettore `val_actual`
    - `zip(*P_val.nonzero())` restituisce tutte le tuple `(u, i)` per cui `P_val[u, i]` è `True`

In [40]:
def get_val_predictions(pred_func):
    return np.array([
        pred_func(u, i)
        for u, i in zip(*P_val.nonzero())
    ])

## RMSE

- Il _Root Mean Squared Error_ (RMSE) è una metrica di valutazione usata comunemente per i sistemi di recommendation
- Dato un validation set $V$ di voti reali $r_{u,i}$ per cui abbiamo estratto delle predizioni $\hat{r}_{u,i}$, il RMSE è
$$ \text{RMSE} = \sqrt{\frac{1}{|V|} \sum_{r_{u,i}\in V} (\hat{r}_{u,i}-r_{u,i})^2} $$
- Si può usare per comparare diversi modelli di recommendation sugli stessi dati
  - essendo una misura d'errore, i modelli migliori sono quelli con RMSE minore
- Similmente al MSE dei modelli di regressione, non è facile interpretare quanto sia "buona" senza un riferimento
  - si può ad es. comparare a recommendation casuali, che vedremo come estrarre con Surprise

### Esercizio 4: Calcolo del RMSE

- **(4a)** Definire una funzione `RMSE` che, dati dei vettori di voti reali e corrispondenti predizioni, calcoli e restituisca il RMSE
- **(4b)** Usando `get_val_predictions`, estrarre i voti predetti sul validation set dalla funzione `predict_from_all`
- **(4c)** Calcolare il RMSE di tali predizioni

In [41]:
def RMSE(actual, predicted):
    return np.sqrt( ((actual-predicted)**2).sum() / actual.shape[0] )

In [42]:
val_pred = get_val_predictions(predict_from_all)

<ipython-input-27-e82a6ea34d9b>:3: RuntimeWarning: invalid value encountered in double_scalars
  predicted_vote = (cosim[u, voters] @ R[voters, i]) / cosim[u, voters].sum()


In [43]:
RMSE(val_actual, val_pred)

1.1764549883655138

## Selezione degli utenti simili

- Nella versione base, abbiamo predetto ciascun voto sulla base di tutti gli altri utenti che hanno acquistato l'oggetto
- Per migliorare l'efficienza e potenzialmente l'accuratezza delle predizioni, possiamo limitarci ad un numero $k$ di utenti, selezionando quelli più simili
- Per ogni coppia utente-oggetto $(u,i)$, consideriamo un _vicinato_ $N^k_i(u)$ degli utenti più simili a $u$ che hanno acquistato $i$
- La predizione è calcolata come sopra, ma su questo vicinato invece che su tutti gli utenti che hanno acquistato $i$
$$ \hat{r}_{u,i} = \frac{\sum_{v \in N^k_i(u)} \text{sim}(u, v) \cdot r_{v,i}}{\sum_{v \in N^k_i(u)} \text{sim}(u, v)} $$
- Impostiamo un valore per il parametro $k$, ad esempio:

In [44]:
k = 10

### Esercizio 5: Calcolo predizioni su utenti simili

- **(5a)** Completare la definizione della funzione `predict_from_neighbors` in modo che predica un voto $\hat{r}_{u,i}$ sulla base dei $k$ più simili tra gli utenti che hanno acquistato $i$
  - copiare l'implementazione dalla funzione `predict_from_all` dell'esercizio 3
  - modificarla in modo da ordinare la lista di indici `voters` per similarità ad $u$ e selezionare i primi $k$
- **(5b)** Estrarre con questa funzione i voti predetti sul validation set ed estrarne il RMSE

In [45]:
def predict_from_neighbors(u, i):
    voters = list(P[:, i].nonzero()[0])
    voters.sort(key=lambda x: cosim[u, x], reverse=True)
    voters = voters[:k]
    predicted_vote = (cosim[u, voters] @ R[voters, i]) / cosim[u, voters].sum()
    return predicted_vote if not np.isnan(predicted_vote) else R.mean(where=P)

In [46]:
RMSE(val_actual, get_val_predictions(predict_from_neighbors))

<ipython-input-45-4fd62ffc4afa>:5: RuntimeWarning: invalid value encountered in double_scalars
  predicted_vote = (cosim[u, voters] @ R[voters, i]) / cosim[u, voters].sum()


1.17679042122965

## Surprise

- _Surprise_ è una libreria Python per la creazione e la validazione di modelli di recommendation
  - definisce strutture per rappresentare i dati su cui addestrare i modelli
  - permette di caricare dati da diverse fonti o di utilizzare dataset d'esempio
  - implementa diverse tecniche basate su similarità, scomposizione di matrici, ...
  - fornisce funzionalità per validare i modelli calcolando comuni metriche di accuratezza come il RMSE

## Dataset

- Un oggetto `Dataset` consiste in un insieme di voti conosciuti dati da degli utenti a degli oggetti
- Utenti e oggetti in un `Dataset` sono rappresentati con identificatori arbitrari scelti dall'utente, spesso numeri o stringhe
  - questi sono chiamati identificatori _raw_ in Surprise
- Un `Dataset` può essere ottenuto da un file CSV o da un DataFrame pandas (a sua volta ottenibile da diverse fonti)
  - Surprise permette inoltre di caricare diversi dataset d'esempio di uso comune, scaricati _on demand_ dal Web

### Caricamento Dataset da CSV

- Per caricare dati da file CSV, dobbiamo prima creare un oggetto `Reader` che indichi le caratteristiche dei file
  - con `sep` indichiamo il separatore di campo usato nei CSV
  - con `rating_scale` indichiamo la scala di voti adottata, in forma di tupla con valori minimo e massimo

In [47]:
from surprise import Reader
csv_reader = Reader(sep=",", rating_scale=(1, 5))

- Usiamo quindi la funzione `load_from_file` per caricare i due dataset, indicando i nomi dei file e il `Reader` da usare

In [48]:
from surprise import Dataset
train_dataset = Dataset.load_from_file("amazon_train.csv", csv_reader)
val_dataset = Dataset.load_from_file("amazon_val.csv", csv_reader)

## Trainset

- Un `Trainset` contiene le stesse informazioni di un `Dataset` in forma ottimizzata per l'utilizzo da parte degli algoritmi d'apprendimento
- $M$ utenti ed $N$ oggetti distinti sono identificati in un `Trainset` da identificatori interni (_inner_), ovvero numeri sequenziali da 0 a $M-1$ e da 0 a $N-1$
  - il `Trainset` tiene comunque traccia delle corrispondenze tra ID raw e inner
- Per creare un `Trainset` con tutti i voti contenuti in un `Dataset`, usare il metodo `build_full_trainset` di quest'ultimo

In [49]:
trainset = train_dataset.build_full_trainset()

### Estrarre Informazioni da un Trainset

- Da un `Trainset` abbiamo accesso rapido ad informazioni quali il numero di utenti distinti, oggetti distinti e voti complessivi

In [50]:
trainset.n_users, trainset.n_items, trainset.n_ratings

(178, 3384, 9683)

- Possiamo consultare l'elenco dei voti noti di un qualsiasi utente od oggetto dai dizionari `ur` e `ir`
  - vanno usati gli ID seriali interni dal `Trainset`

In [51]:
# es.: 3 voti dati dall'utente 0
trainset.ur[0][:3]
# ogni tupla: (ID oggetto, voto)

[(0, 2.0), (1, 3.0), (69, 3.0)]

- Possiamo vedere la media globale di tutti i voti dati

In [52]:
trainset.global_mean

3.931219663327481

## Addestramento di un modello user-based

- In Surprise possiamo addestrare modelli di recommendation, in modo simile ai modelli di regressione in scikit-learn
  - un modello viene creato, specificandone eventuali iperparametri
  - il modello viene addestrato sui dati del training set
  - il modello può essere usato per estrarre predizioni o validato su un validation set
- La recommendation user-based è implementata nella classe `KNNBasic`
  - con `k` specifichiamo il numero di utenti simili da considerare
  - con un dizionario `sim_options` impostiamo la misura di similarità
- Creiamo ad esempio un modello come il primo creato sopra, che usi la similarità coseno e consideri sempre tutti gli utenti per ogni predizione
  - per non limitare gli utenti, poniamo `k` uguale al numero totale di utenti

In [53]:
from surprise import KNNBasic
ubr = KNNBasic(k=trainset.n_users, sim_options={"name": "cosine"})

- Per eseguire l'addestramento del modello, chiamarne il metodo `fit` passando il `Trainset` da utilizzare

In [54]:
ubr.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


- Con l'addestramento viene costruita la matrice delle similarità coseno, accessibile dall'attributo `sim`
  - questa può non essere identica alla matrice `cosim` estratta sopra manualmente, per via degli utenti ordinati diversamente

In [ ]:
ubr.sim[:4, :4]

array([[1.        , 0.94299033, 1.        , 0.94255773],
       [0.94299033, 1.        , 1.        , 0.98      ],
       [1.        , 1.        , 1.        , 0.97646729],
       [0.94255773, 0.98      , 0.97646729, 1.        ]])

## Utilizzare un modello

- Una volta addestrato, il modello può prevedere il rating che un utente darebbe ad un oggetto dati i rispettivi ID "raw", cioè quelli usati nel `Dataset` originale
- Ad esempio, estraiamo i nomi dell'utente e dell'oggetto che avevano indice 0 nella prima parte

In [55]:
user0 = train_ratings.index[0]
item0 = train_ratings.columns[0]

- Passiamo i nomi al metodo `predict` per ottenere la predizione da parte del modello surprise

In [56]:
pred = ubr.predict(user0, item0)

- Otteniamo un oggetto `Prediction` i cui attributi riepilogano la richiesta (`uid` e `iid`) e forniscono i dati della predizione

In [57]:
pred

Prediction(uid='[1001561] D. Wetzel', iid='[100004] High School Confidential [VHS]', r_ui=None, est=5, details={'actual_k': 1, 'was_impossible': False})

- Il voto predetto è dato dall'attributo `est`, in questo caso 5 stelle
- I `details` indicano informazioni aggiuntive, in questo caso che il voto è stato predetto sulla base di quello dato da 1 solo utente simile
- `r_ui` è il voto reale (in questo caso non c'è, ma lo vedremo a breve)
- Possiamo accedere a tutti i dati come attributi dell'oggetto, ad es.:

In [ ]:
pred.est

5

- Possiamo verificare che questa predizione è la stessa fatta dal modello costruito manualmente sopra

In [ ]:
predict_from_all(0, 0)

5.0

## Validare un modello

- Per eseguire la validazione del modello in surprise dobbiamo
  - costruire un validation set in forma di una lista di tuple `(utente, oggetto, voto)`
  - fornire questa lista al modello da validare per ottenere una lista di `Prediction` corrispondenti
  - calcolare una metrica di accuratezza (es. RMSE) su tale lista
- Per ottenere un validation set nella forma corretta, possiamo usare il metodo `build_testset`:

In [58]:
valset = val_dataset.build_full_trainset().build_testset()

In [59]:
valset[:3]

[('[1092996] Reviewer', '[4742] Nights of Cabiria [VHS]', 5.0),
 ('[1092996] Reviewer', '[50377] The Quiet Man [VHS]', 5.0),
 ('[1092996] Reviewer', '[51373] Home Alone 2 - Lost in New York [VHS]', 3.0)]

- Tale lista va passata al metodo `test` del modello addestrato

In [60]:
preds = ubr.test(valset)

- Otteniamo una lista di oggetti `Prediction`

In [ ]:
preds[:3]

[Prediction(uid='[1092996] Reviewer', iid='[4742] Nights of Cabiria [VHS]', r_ui=5.0, est=5, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid='[1092996] Reviewer', iid='[50377] The Quiet Man [VHS]', r_ui=5.0, est=4.241931710095136, details={'actual_k': 4, 'was_impossible': False}),
 Prediction(uid='[1092996] Reviewer', iid='[51373] Home Alone 2 - Lost in New York [VHS]', r_ui=3.0, est=3.0, details={'actual_k': 1, 'was_impossible': False})]

- In ciascun oggetto troviamo rispettivamente in `r_ui` e in `est` i voti reali e predetti

- Possiamo quindi passare questa lista ad una funzione per il calcolo di una metrica di accuratezza
- Calcoliamo ad esempio il RMSE come abbiamo fatto sopra, verificando che combaci

In [61]:
from surprise.accuracy import rmse
rmse(preds)

RMSE: 1.1765


1.1764549883655138

- Un'altra metrica che può essere calcolata è il MAE (_mean absolute error_), la media degli errori in valore assoluto

In [62]:
from surprise.accuracy import mae
mae(preds)

MAE:  0.8526


0.8525568703741262

### Esercizio 6: Modello user-based con vicinato

- **(6a)** Addestrare sul training set un modello di recommendation user-based basato su similarità coseno e vicinato di 10 utenti, come quello costruito manualmente nella prima parte
- **(6b)** Calcolare sul validation set il RMSE e il MAE di tale modello
  - potrebbe risultare lievemente diverso da quello calcolato in precedenza per via di diversi utenti vicini selezionati nei casi di pari similarità

In [65]:
mdl = KNNBasic(k=10, sim_options={"name": "cosine"})
mdl.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [66]:
preds = mdl.test(valset)
rmse(preds)
mae(preds)

RMSE: 1.1770
MAE:  0.8525


0.8525394518396882

## Varianti sul modello user-based

- Nel modello user-based è possibile utilizzare altre misure di similarità diverse dal coseno
- Una scelta comune è la _correlazione di Pearson_
  - in pratica consiste nella similarità coseno misurata non sui voti ma sul loro scarto rispetto alla media
  - questo bilancia le tendenze di voto diverse degli utenti, equiparando ad es. le 3 stelle di un utente alle 2 di uno più "severo"
- Per utilizzarla, specifichiamola in `sim_options` al posto del coseno

In [ ]:
ubr = KNNBasic(k=10, sim_options={"name": "pearson"})

- Possiamo quindi addestrare e validare il modello come prima

In [ ]:
ubr.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [ ]:
preds = ubr.test(valset)
rmse(preds), mae(preds)

RMSE: 1.1833
MAE:  0.8782


(1.1833093002514359, 0.8782414100948562)

- In questo caso l'uso della correlazione di Pearson non ha migliorato il risultato

- Un altro modo per tenere conto delle differenze di voto medio tra gli utenti è riconsiderare il modo in cui è stimato il voto
- Invece di stimare direttamente il voto di $u$ come media di altri voti, possiamo stimare lo _scostamento dalla media_ del voto di $u$ come media degli scostamenti dalle rispettive medie di altri utenti
$$ \hat{r}_{u,i} = \bar{r_u}+\frac{\sum_{v:P_{v,i}=1} \text{sim}(u, v) \cdot (r_{v,i}-\bar{r_v})}{\sum_{v:P_{v,i}=1} \text{sim}(u, v)} $$
- Questo principio viene applicato nella classe `KNNWithMeans`, che per il resto è identica a `KNNBasic` e accetta le stesse opzioni

In [67]:
from surprise import KNNWithMeans
ubr = KNNWithMeans(k=10, sim_options={"name": "cosine"})

- Addestriamo e validiamo il modello come al solito

In [68]:
ubr.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [69]:
preds = ubr.test(valset)
rmse(preds), mae(preds)

RMSE: 1.0494
MAE:  0.7699


(1.049420754780564, 0.7698569855396796)

- Otteniamo un RMSE migliore

## Collaborative filtering item-based

- La classe `KNNBasic` può essere usata anche per eseguire recommendation item-based, metodo duale allo user-based
- Il voto $\hat{r}_{ui}$ previsto per un oggetto $i$ da parte di un utente $u$ è dato dalla media pesata dei voti dati da $u$ ai $k$ oggetti più simili ad $i$, rappresentati in un insieme $N_u^k(i)$
$$ \hat{r}_{ui} = \frac{\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot r_{uj}}{\sum\limits_{j \in N^k_u(j)} \text{sim}(i, j)} $$
- La similarità $\text{sim}(i,j)$ tra due oggetti $i$ e $j$ è misurata dai voti ricevuti da utenti che hanno recensito entrambi
- Per eseguire item-based recommendation al posto di user-based, impostiamo `user_based=False` nelle opzioni della misura di similarità:

In [77]:
ibr = KNNBasic(k=10, sim_options={"name": "cosine", "user_based": False})

- Possiamo quindi addestrare e validare il modello come prima

In [78]:
ibr.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [79]:
preds = ibr.test(valset)
rmse(preds), mae(preds)

RMSE: 1.1381
MAE:  0.8494


(1.1381231208660862, 0.849433685290423)

## Collaborative filtering con fattorizzazione di matrici

I metodi basati su fattorizzazione o scomposizione di matrici funzionano rappresentando utenti ed oggetti come combinazione di fattori
- i fattori sono ricavati statisticamente dai dati e corrispondono a grandi linee a categorie di oggetti (es. per i film: azione, commedia, ...)
- ciascun oggetto è rappresentato da un vettore col peso di ciascun fattore su di esso (es. quanto un film è d'azione)
- il vettore di ciascun utente indica l'affinità a ciascun fattore (es. quanto gli piacciono i film d'azione)
- il voto stimato è quindi proporzionale alla similarità tra il vettore dell'utente e quello del prodotto

- Il modello più semplice con fattorizzazione di matrici è `SVD` (_singular value decomposition_)
  - l'addestramento avviene internamente tramite una discesa gradiente _stocastica_, ovvero in cui i gradienti sono calcolati per efficienza su campioni casuali di dati
  - il parametro principale è il numero di fattori da individuare `n_factors`
  - con `n_epochs` si può invece controllare il numero di iterazioni di discesa gradiente
  - con `random_state` si può fornire un seed per la componente stocastica (necessario per la riproducibilità)

In [74]:
from surprise import SVD
fbr = SVD(n_factors=10, random_state=42)

In [75]:
fbr.fit(trainset)

In [76]:
preds = fbr.test(valset)
rmse(preds), mae(preds)

RMSE: 0.9565
MAE:  0.7446


(0.9565151460041259, 0.7445630800807548)

- Il modello SVD fornisce risultati migliori di quelli sopra

## Recommendation casuale

- Surprise offre anche un recommender `NormalPredictor` che prevede voti casuali, utilizzabile come baseline nella valutazione degli altri metodi
  - i voti predetti hanno una distribuzione normale con media e varianza calcolate dal training set

In [72]:
from surprise import NormalPredictor
rr = NormalPredictor()
rr.fit(trainset)

In [73]:
preds = rr.test(valset)
rmse(preds), mae(preds)

RMSE: 1.5218
MAE:  1.1891


(1.5218202092809758, 1.18910963487458)

- Estraendo i voti a caso ad ogni chiamata, la valutazione eseguita più volte dà risultati diversi
  - per impostare un seed prima dell'addestramento: `np.random.seed(123)`
- Ovviamente questo metodo è molto veloce ma ha un RMSE molto più alto

## Cross validation k-fold

- Per eseguire la validazione dei modelli provati, abbiamo fin quì usato il metodo _hold-out_, cioè la divisione tra un training e un validation set predefiniti
- Come visto nei modelli di regressione, possiamo ottenere una validazione più accurata con la cross validation a _k_ fold
  - il dataset è suddiviso casualmente in _k_ gruppi, ciascuno è usato come validation set di un modello addestrato su tutti gli altri
- Surprise fornisce funzionalità per la cross validation con un API molto simile a quella di scikit-learn
- Per definire come suddividere il dataset, creiamo un oggetto `KFold`
  - specifichiamo il numero di fold da creare e un seed per la casualità

In [70]:
from surprise.model_selection import KFold
kf = KFold(n_splits=3, shuffle=True, random_state=42)

- Per eseguire una cross validation possiamo usare la funzione `cross_validate` indicando
  - il modello (algoritmo e parametri) su cui eseguire la validazione
  - il `Dataset` da utilizzare
  - il criterio di split da usare, ad es. il `KFold` sopra
  - le misure di accuratezza da calcolare (default: RMSE e MAE)
- Prendiamo ad esempio il modello user-based basato su 10 vicini e similarità coseno visto prima

In [71]:
from surprise.model_selection import cross_validate
model = KNNBasic(k=10, sim_options={"name": "cosine"})
cv_results = cross_validate(model, train_dataset, cv=kf)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


- Come in scikit-learn, otteniamo un dizionario che possiamo visualizzare comodamente in un frame pandas

In [ ]:
pd.DataFrame(cv_results)

,test_rmse,test_mae,fit_time,test_time
0,1.254324,0.951606,0.007901,0.026748
1,1.231866,0.926686,0.008215,0.026943
2,1.251063,0.937161,0.009153,0.026701


- Ogni riga corrisponde ad un fold
- Le colonne `fit_time` e `test_time` riportano il tempo (in secondi) richiesto per addestramento e validazione
- Le colonne `test_rmse` e `test_mae` riportano le misure di accuratezza
- Possiamo usare le medie dei valori come valutazioni generali del modello testato sui dati

In [ ]:
cv_results["test_rmse"].mean()

1.2457509255127395

## Grid search

- Come i modelli di regressione, anche quelli di recommendation hanno degli iperparametri da impostare che possono influenzarne l'accuratezza
  - ad esempio il numero k di vicini nella recommendation user-based
- Surprise offre una funzionalità di _grid search_ simile a quella di scikit-learn per testare diversi valori dei parametri
- Vediamo ad esempio l'efficacia della user-based recommendation al variare di k e della misura di similarità
- Si definisce una "griglia" con i valori possibili dei parametri

In [80]:
grid = {
    "k": [5, 10, 20],
    "sim_options": {"name": ["cosine", "pearson"]}
}

- Si crea quindi un oggetto `GridSearchCV` passando il modello da testare, la griglia dei parametri e il metodo di split per la validazione
  - Attenzione! Differentemente da scikit-learn va passata _la classe_ del modello da usare (in questo caso `KNNBasic`), non una sua istanza
  - se la classe ha dei parametri con valori fissi, questi possono essere inseriti nella griglia insieme a quelli variabili
  - con `refit=True` specifichiamo che alla fine va riaddestrato su tutti i dati un modello con i parametri risultati migliori (di default basandosi sul RMSE)

In [81]:
from surprise.model_selection import GridSearchCV
gs = GridSearchCV(KNNBasic, grid, cv=kf, refit=True)

- Si chiama quindi il metodo `fit` passando il `Dataset` con i dati

In [82]:
gs.fit(train_dataset)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Comput

- L'oggetto addestrato può essere ispezionato in modo simile a scikit-learn
- `best_params` fornisce la combinazione di parametri risultata migliore per ciascuna metrica di accuratezza

In [83]:
gs.best_params

{'rmse': {'k': 10, 'sim_options': {'name': 'pearson', 'user_based': True}},
 'mae': {'k': 10, 'sim_options': {'name': 'cosine', 'user_based': True}}}

- `cv_results` fornisce tutti i risultati, può essere visualizzato come frame con le righe ordinate per RMSE o MAE medio

In [84]:
pd.DataFrame(gs.cv_results).sort_values("mean_test_rmse")

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_k,param_sim_options
3,1.255051,1.227805,1.233077,1.238644,0.011799,1,0.967408,0.946476,0.942974,0.952286,0.010788,4,0.004071,0.000931,0.036021,0.003037,"{'k': 10, 'sim_options': {'name': 'pearson', '...",10,"{'name': 'pearson', 'user_based': True}"
5,1.255065,1.227821,1.233085,1.238657,0.011800,2,0.967457,0.946491,0.942922,0.952290,0.010823,5,0.004061,0.000756,0.034097,0.000998,"{'k': 20, 'sim_options': {'name': 'pearson', '...",20,"{'name': 'pearson', 'user_based': True}"
1,1.255666,1.227754,1.233733,1.239051,0.011999,3,0.968043,0.946195,0.943654,0.952631,0.010948,6,0.003799,0.000494,0.036786,0.002536,"{'k': 5, 'sim_options': {'name': 'pearson', 'u...",5,"{'name': 'pearson', 'user_based': True}"
4,1.252696,1.231326,1.249788,1.244603,0.009463,4,0.951588,0.927287,0.937037,0.938637,0.009985,2,0.003453,0.000668,0.039895,0.004847,"{'k': 20, 'sim_options': {'name': 'cosine', 'u...",20,"{'name': 'cosine', 'user_based': True}"
2,1.254324,1.231866,1.251063,1.245751,0.009908,5,0.951606,0.926686,0.937161,0.938484,0.010217,1,0.003645,0.000895,0.038439,0.002787,"{'k': 10, 'sim_options': {'name': 'cosine', 'u...",10,"{'name': 'cosine', 'user_based': True}"
0,1.261489,1.239035,1.255559,1.252028,0.009501,6,0.953364,0.930041,0.938673,0.940693,0.009628,3,0.003131,0.000352,0.039902,0.000593,"{'k': 5, 'sim_options': {'name': 'cosine', 'us...",5,"{'name': 'cosine', 'user_based': True}"


- Possiamo utilizzare l'oggetto come modello, validandolo ad esempio sui dati non utilizzati del validation set

In [85]:
preds = gs.test(valset)
rmse(preds), mae(preds)

RMSE: 1.1833
MAE:  0.8782


(1.1833093002514359, 0.8782414100948562)

### Esercizio 7: Grid search

- **(7a)** Con una grid search, usando stessi dati e criterio di split usati sopra, individuare il valore ottimale (in termini di RMSE) del parametro `n_factors` per un modello `SVD`, testando i valori 3, 6, 9, ..., 30
- **(7b)** Calcolare RMSE e MAE sul validation set del modello migliore individuato

In [95]:
grid = {
    "n_factors": range(3,31,3),
}
gs = GridSearchCV(SVD, grid, cv=kf, refit=True)
gs.fit(train_dataset)
gs.best_params["rmse"]["n_factors"]

6

In [96]:
preds = gs.test(val_dataset.build_full_trainset().build_testset())
rmse(preds)
mae(preds)

RMSE: 0.9554
MAE:  0.7437


0.7437176184406784

## Esercizio 8: Suggerire prodotti agli utenti

Un modello di recommendation è in genere utilizzato, basandosi sui voti predetti, per suggerire un numero di prodotti ad un dato utente

- **(8a)** Usando il modello ottenuto nell'esercizio 7, estrarre una lista di `Prediction` con i voti predetti per l'utente `target_user` definito sotto su tutti i prodotti nel training set
  - per scorrere tutti i prodotti, scorrere gli indici del `trainset` da 0 a `n_items-1` e usare `trainset.to_raw_iid` per ottenere i nomi corrispondenti
  - per semplicità, non è necessario escludere i prodotti già valutati
- **(8b)** Stampare i nomi dei 10 prodotti con voto predetto più alto
- **(8c)** Definire una funzione `recommend` che, dato un nome utente e un numero di prodotti da suggerire, restituisca una lista di nomi di prodotti suggeriti ottenuta come nei punti sopra

In [97]:
target_user = "[624547] Roland E. Zwick"

In [110]:
predictions = [gs.predict(target_user, trainset.to_raw_iid(ii)) for ii in trainset.all_items()]
predictions.sort(key=lambda x: x.est, reverse=True)
for item in predictions[:10]:
  print(item.iid)

[7989] Fight Club
[60001] The Iron Giant
[95693] The Shawshank Redemption (Widescreen Edition) [VHS]
[44091] The Wizard of Oz
[103868] 2001: A Space Odyssey [VHS]
[50060] Nosferatu [VHS]
[8039] Boys Don't Cry [VHS]
[3574] Halloween
[95853] Schindler's List (Widescreen Edition) [VHS]
[4773] Life Is Beautiful (La Vita E Bella) [VHS]


In [118]:
def recommend(user, p_number):
    return [i.iid for i in sorted([gs.predict(target_user, trainset.to_raw_iid(ii)) for ii in trainset.all_items()], key=lambda x: x.est, reverse=True)[:p_number]]

In [120]:
recommend(target_user, 50)

['[7989] Fight Club',
 '[60001] The Iron Giant',
 '[95693] The Shawshank Redemption (Widescreen Edition) [VHS]',
 '[44091] The Wizard of Oz',
 '[103868] 2001: A Space Odyssey [VHS]',
 '[50060] Nosferatu [VHS]',
 "[8039] Boys Don't Cry [VHS]",
 '[3574] Halloween',
 "[95853] Schindler's List (Widescreen Edition) [VHS]",
 '[4773] Life Is Beautiful (La Vita E Bella) [VHS]',
 '[96662] Platoon [VHS]',
 '[43989] Braveheart [VHS]',
 '[47806] The Omen [VHS]',
 "[42911] Das Boot - The Director's Cut",
 '[96039] Terminator 2: Judgment Day',
 '[51262] Glengarry Glen Ross [VHS]',
 '[96741] Glory',
 '[104083] Tootsie [VHS]',
 '[43736] The Fugitive',
 '[96732] The Crucible [VHS]',
 '[95843] Jaws [VHS]',
 '[44088] Poltergeist',
 '[95656] Boogie Nights',
 '[97727] Pulp Fiction',
 '[43426] 12 Monkeys (Widescreen Edition) [VHS]',
 '[47818] Aliens [VHS]',
 '[1804] The Usual Suspects Letterbox Edition  [VHS]',
 '[49432] A Christmas Story [VHS]',
 '[57780] Titus',
 '[47792] Alien [VHS]',
 "[43463] The Thing